<a href="https://colab.research.google.com/github/Enrico-Call/RL-AKI/blob/modeling-notebook/Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/master/img/logo_amds.png?raw=1" alt="Logo" width=128px/>

# VUmc Research Project - Reinforcement Learning for Sepsis Prevention
# Data Extraction

AmsterdamUMCdb version 1.0.2 March 2020  
Copyright &copy; 2003-2022 Amsterdam UMC - Amsterdam Medical Data Science

## 1. Set Up Environment and Load Data 

In [ ]:
from google.colab import drive

import os
import pandas as pd

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/MLRFH')

state_space = pd.read_csv('.csv')
action_space = pd.read_csv('.csv')

## 2. Modeling

Create a Q-Learning Agent to iterate over the dataset.

In [ ]:
# # Transition Matrix

# # Counting number of unique clusters (now we know it, but for later if we add more might be useful)

# numbers = sorted(final['cluster'].unique())

# # Find how many times a state is followed by another

# groups = final.groupby(['cluster', 'next'])
# counts = {i[0]:(len(i[1]) if i[0][0] != i[0][1] else 0) for i in groups} 

# # Build a matrix based on the counts just performed

# matrix = pd.DataFrame()

# for x in numbers:
#     matrix[x] = pd.Series([counts.get((x,y), 0) for y in numbers], index=numbers)
  
# matrix_normed = matrix / matrix.sum(axis=0)
# matrix = matrix_normed.to_numpy()
# print(matrix)

In [ ]:
# Check how many times a state is associated with a certain reward

# For now, it might make sense to model the reward of each state based on the highest number of occurrences of a certain reward

# rewards = final.groupby(['cluster', 'reward']).size()
# print(rewards)

rewards = final.groupby(['cluster'], sort=False)['reward'].max()
rewards.sort_index(inplace=True)
rewards = rewards.to_numpy()

In [ ]:
# rew = final.groupby(['cluster', 'action', 'next']).size()
# # rew.sort_index(inplace=True)
# print(rew)
# # rew = rew.to_numpy()

In [ ]:
# alphas = range(0, 85, 5)  # Learning Rate
# gamma = 0.99 # Discount Factor

# for alpha in alphas:

#   alpha = alpha/100
#   val = []
  
#   # Q(s, a) matrix

#   for i in range(10):

#     Q = np.zeros((50, 4))

#     from sklearn.model_selection import GroupShuffleSplit

#     train_inds, test_inds = GroupShuffleSplit(test_size=.20, n_splits=2, random_state = 7).split(final, groups=final['admissionid'])

#     train = final.iloc[train_inds[0]]
#     test = final.iloc[test_inds[0]]

#     iterations = 1000000

#     train = train[['cluster', 'action', 'next', 'reward']].sample(n = iterations, replace = True)
#     test = test[['cluster', 'action', 'next', 'reward']].sample(n = iterations, replace = True)
#     i = 0

#     for row in train.loc[:,['cluster', 'action', 'next', 'reward']].itertuples():
#       index, curr, act, next, rew = row
#       delta = rew+gamma*np.max(Q[int(next), :])- Q[int(curr), int(act)]
#       Q[int(curr), int(act)] += alpha*delta
#       i += 1
#       # if the update is smaller than a certain threshold, stop the iteration

#       # if delta < 1e-10: break

#     p_optim = 0.9

#     actions = np.argmax(Q, axis = 1)
#     pi = np.full((50, 4), ((1-p_optim)/(50-1)))
#     for i, j in enumerate(actions):
#       pi[i,j] = p_optim

#     Q_1 = np.zeros((50, 4))

#     for row in test.loc[:,['cluster', 'action', 'next', 'reward']].itertuples():
#       index, curr, act, next, rew = row
#       Q_1[int(curr), int(act)] += 1

#     Q_1 = Q_1/Q_1.sum(axis=1, keepdims=True)

#     best_act_pi = np.argmax(Q, axis=1)
#     best_act_test = np.argmax(Q_1, axis=1)

#     val.append(np.count_nonzero(best_act_pi == best_act_test))

#   val = np.array(val)
#   print(alpha, val.mean())

### Parameter Tuning

In [ ]:
alphas = range(0, 85, 5)  # Learning Rate
gamma = 0.99 # Discount Factor

for alpha in alphas:

  alpha = alpha/100

  Q = np.zeros((50, 4))

  from sklearn.model_selection import GroupShuffleSplit

  train_inds, test_inds = GroupShuffleSplit(test_size=.20, n_splits=2, random_state = 7).split(final, groups=final['patientid'])

  train = final.iloc[train_inds[0]]
  test = final.iloc[test_inds[0]]

  iterations = 1000000

  train = train[['cluster', 'action', 'next', 'reward']].sample(n = iterations, replace = True)
  test = test[['cluster', 'action', 'next', 'reward']].sample(n = iterations, replace = True)
  i = 0

  for row in train.loc[:,['cluster', 'action', 'next', 'reward']].itertuples():
    index, curr, act, next, rew = row
    delta = rew+gamma*np.max(Q[int(next), :])- Q[int(curr), int(act)]
    Q[int(curr), int(act)] += alpha*delta
    i += 1
    # if the update is smaller than a certain threshold, stop the iteration

    # if delta < 1e-10: break

  p_optim = 0.9

  actions = np.argmax(Q, axis = 1)
  pi = np.full((50, 4), ((1-p_optim)/(50-1)))
  for i, j in enumerate(actions):
    pi[i,j] = p_optim

  Q_1 = np.zeros((50, 4))

  for row in test.loc[:,['cluster', 'action', 'next', 'reward']].itertuples():
    index, curr, act, next, rew = row
    Q_1[int(curr), int(act)] += 1

  Q_1 = Q_1/Q_1.sum(axis=1, keepdims=True)

  best_act_pi = np.argmax(Q, axis=1)
  best_act_test = np.argmax(Q_1, axis=1)

  print(alpha, np.count_nonzero(best_act_pi == best_act_test))

### Run Model

In [ ]:
from sklearn.model_selection import GroupShuffleSplit

train_inds, test_inds = GroupShuffleSplit(test_size=.20, n_splits=2, random_state = 7).split(final, groups=final['admissionid'])

train = final.iloc[train_inds[0]]
test = final.iloc[test_inds[0]]

train = train[['cluster', 'action', 'next', 'reward']].sample(n = iterations, replace = True)
test = test[['cluster', 'action', 'next', 'reward']].sample(n = iterations, replace = True)

alphas = 0.75  # Learning Rate
gamma = 0.99 # Discount Factor

avg_Q = []
avg_Q1 = []

for i in range(50):

  Q = np.zeros((50, 4))

  iterations = 1000000

  i = 0

  for row in train.loc[:,['cluster', 'action', 'next', 'reward']].itertuples():
    index, curr, act, next, rew = row
    delta = rew+gamma*np.max(Q[int(next), :])- Q[int(curr), int(act)]
    Q[int(curr), int(act)] += alphas*delta
    i += 1
    # if the update is smaller than a certain threshold, stop the iteration

    # if delta < 1e-10: break

  p_optim = 0.9

  actions = np.argmax(Q, axis = 1)
  pi = np.full((50, 4), ((1-p_optim)/(50-1)))
  for i, j in enumerate(actions):
    pi[i,j] = p_optim

  avg_Q.append(np.average(pi, axis=0)) 

In [ ]:
avg_Q1 = []

for i in range(10):
  Q_1 = np.zeros((50, 4))

  for row in test.loc[:,['cluster', 'action', 'next', 'reward']].itertuples():
    index, curr, act, next, rew = row
    Q_1[int(curr), int(act)] += 1

  Q_1 = Q_1/Q_1.sum(axis=1, keepdims=True)

  best_act_pi = np.argmax(Q, axis=1)
  best_act_test = np.argmax(Q_1, axis=1)

  avg_Q1.append(np.average(Q_1, axis = 0))

In [ ]:
Q_plot = np.average(pi, axis=0)
Q_1_plot = np.average(Q_1, axis = 0)

Q_plot, Q_1_plot

In [ ]:
np.count_nonzero(best_act_pi == best_act_test)

In [ ]:
labels = ['0', '1', '2', '3']

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, Q_plot, width, label='Algorithm Policy')
rects2 = ax.bar(x + width/2, Q_1_plot, width, label='Test Set Actions')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Probability of Taking Action')
ax.set_title('Action')
ax.set_xticks([0, 1, 2, 3])
ax.legend()

fig.tight_layout()

plt.show()